In [1]:
import requests
import json

### Test health check

In [2]:
url = "http://localhost:9000/2015-03-31/functions/function/invocations"

payload = {
    "resource": "/healthcheck",
    "path": "/healthcheck",
    "httpMethod": "GET",
    "requestContext": {
        },
    "isBase64Encoded": False
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(url, data=json.dumps(payload), headers=headers)

In [3]:
print(response.json())

{'statusCode': 200, 'headers': {'content-length': '95', 'content-type': 'application/json'}, 'multiValueHeaders': {}, 'body': '{"message":"The server is up since 2023-08-12 03:57:28","start_uct_time":"2023-08-12 03:57:28"}', 'isBase64Encoded': False}


```sh
# Same as 
curl -XPOST "http://localhost:9000/2015-03-31/functions/function/invocations" -d '{
    "resource": "/healthcheck",
    "path": "/healthcheck",
    "httpMethod": "GET",
    "requestContext": {
    },
    "isBase64Encoded": false
}'
```


### Test Recommend Endpoint 

In [7]:
event = {'movie_ids':[1,2,3,4],
'user_age':23,
"sex":"M",
"topk":5}

In [8]:
payload = {
    "resource": "/recommend",
    "path": "/recommend",
    "httpMethod": "POST",
    "requestContext": {
        "resourcePath": "/recommend",
        "httpMethod": "POST"
        },
        "body": json.dumps(event),
        "isBase64Encoded": False
}

headers = {
    "Content-Type": "application/json"
}

In [9]:
response = requests.post(url, data=json.dumps(payload), headers=headers)

In [10]:
print(response.json())

{'statusCode': 200, 'headers': {'content-length': '983', 'content-type': 'application/json'}, 'multiValueHeaders': {}, 'body': '[{"movie_id":318,"title":"Shawshank Redemption, The (1994)","genres":["Drama"],"release_year":1994,"origin_title":"Shawshank Redemption, The","rating":5.0},{"movie_id":858,"title":"Godfather, The (1972)","genres":["Action","Crime","Drama"],"release_year":1972,"origin_title":"Godfather, The","rating":5.0},{"movie_id":2019,"title":"Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954)","genres":["Action","Drama"],"release_year":1954,"origin_title":"Seven Samurai (The Magnificent Seven) (Shichinin no samurai)","rating":5.0},{"movie_id":593,"title":"Silence of the Lambs, The (1991)","genres":["Drama","Thriller"],"release_year":1991,"origin_title":"Silence of the Lambs, The","rating":4.999999523162842},{"movie_id":750,"title":"Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)","genres":["Sci-Fi","War"],"release_year":1963,"or

```sh
# Same AS
curl -XPOST "http://localhost:9000/2015-03-31/functions/function/invocations" -d '{
    "resource": "/recommend",
    "path": "/recommend",
    "httpMethod": "POST",
    "requestContext": {
        "resourcePath": "/recommend",
        "httpMethod": "POST"
    },
    "body": "{\"movie_ids\": [1, 2, 3, 4], \"user_age\": 23, \"sex\": \"M\", \"topk\": 1}",
    "isBase64Encoded": false
}'
```